### Inverse Kinematics
Calculation of the Inverse Kinematics for the **Kuka KR210** using the DH convention

In [33]:
from sympy import symbols, cos, sin, pi, sqrt, simplify, atan2
from sympy.matrices import Matrix
import numpy as np

In [34]:
# Symbolic Variables for oint variables
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') # theta or q
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')

# DH Parameters (taken from the URDF)
s = {alpha0:     0,    a0:      0,   d1:  0.75,
     alpha1: -pi/2,    a1:   0.35,   d2:     0,    q2: q2-pi/2,
     alpha2:     0,    a2:   1.25,   d3:     0,
     alpha3: -pi/2,    a3: -0.054,   d4:  1.50,
     alpha4:  pi/2,    a4:      0,   d5:     0,
     alpha5: -pi/2,    a5:      0,   d6:     0,
     alpha6:     0,    a6:      0,   d7: 0.303,    q7:       0}

In [35]:
### Define functions for Rotation Matrices about x, y, and z given specific angle.
def rot_x(q):
    '''Elementary Rotation Matrix along the X-Axis'''
    R_x = Matrix([[ 1,              0,        0],
                  [ 0,         cos(q),  -sin(q)],
                  [ 0,         sin(q),  cos(q)]])
    return R_x
    
def rot_y(q):  
    '''Elementary Rotation Matrix along the Y-Axis'''
    R_y = Matrix([[ cos(q),         0,  sin(q)],
                  [      0,         1,       0],
                  [-sin(q),         0,  cos(q)]])
    return R_y

def rot_z(q):
    '''Elementary Rotation Matrix along the Z-Axis'''
    R_z = Matrix([[ cos(q), -sin(q),        0],
                  [ sin(q),  cos(q),        0],
                  [ 0,              0,      1]])
    return R_z

### Inverse Position Problem
First solve the inverse position problem that is calculate the first three joint angles that brings the center of the wirst in the desired position

![title](img/ikp.jpg)

Let's call this point W

In [36]:
# Inverse Position Specific Variables - KNOWN in ROS
px, py, pz, roll, pitch, yaw = symbols('px py pz roll pitch yaw') # end effector position and orientation
l = symbols('l') # end effector lengths

# Correction Rotation Matrix that takes into account the change of co
ordinates between the DH and URDF
R_adj = rot_z(pi) * rot_y(-pi/2)

# Resulting Rotation Matrix that express the relation between the base_link and the gripper_link
Rrpy = rot_z(yaw) * rot_y(pitch) * rot_x(roll) * R_adj

# Position of the Wirst Center
Wx = px - (d6 + l) * Rrpy[0, 2]
Wy = py - (d6 + l) * Rrpy[1, 2]
Wz = pz - (d6 + l) * Rrpy[2, 2]

print('Wx: {}'.format(Wx))
print('Wy: {}'.format(Wy))
print('Wz: {}'.format(Wz))

Wx: px - (d6 + l)*cos(pitch)*cos(yaw)
Wy: py - (d6 + l)*sin(yaw)*cos(pitch)
Wz: pz + (d6 + l)*sin(pitch)


Based on the Position of the Wirst Center, calculate the first three joint angles q1, q2, q3

![title](img/ikpp.jpg)

Where in the last column, the first three rows are exactly the position of the Wirst Center

In [37]:
# First Joint Angle - 2 Possibile Configurations
q1_1 = atan2(Wy, Wx) + pi
q1_2 = atan2(Wy, Wx) - pi


# Third Joint Angle - 2 Possibile Configurations
c3 = Wx**2 + Wy**2 + Wz**2 - a2**2 - a3**2
s3 = sqrt(1-(c3**2))

q3_1 = atan2(s3, c3)
q3_2 = atan2(-s3, c3)


# Second Joint Angle - 4 Possible Configurations
c2_1 = (sqrt(Wx**2 + Wy**2)*(a2+a3*c3) + Wz*a3*s3)/(a2**2 + a3**2 + 2*a2*a3*c3)
c2_2 = (-sqrt(Wx**2 + Wy**2)*(a2+a3*c3) + Wz*a3*s3)/(a2**2 + a3**2 + 2*a2*a3*c3)
c2_3 = (sqrt(Wx**2 + Wy**2)*(a2+a3*c3) - Wz*a3*s3)/(a2**2 + a3**2 + 2*a2*a3*c3)
c2_4 = (-sqrt(Wx**2 + Wy**2)*(a2+a3*c3) - Wz*a3*s3)/(a2**2 + a3**2 + 2*a2*a3*c3)
s2_1 = (Wz*(a2 + a3*c3) - sqrt(Wx**2 + Wy**2)*a3*s3)/(a2**2 + a3**2 + 2*a2*a3*c3)
s2_2 = (Wz*(a2 + a3*c3) + sqrt(Wx**2 + Wy**2)*a3*s3)/(a2**2 + a3**2 + 2*a2*a3*c3)

q2_1 = atan2(s2_1, c2_1)
q2_2 = atan2(s2_2, c2_2) 
q2_3 = atan2(s2_2, c2_3)
q2_4 = atan2(s2_1, c2_4)

### Inverse Orientation Problem
Then solve the inverse orientation problem that is calculate the last three joint angles that brings the gripper in the desired orientation.

From the Forward Kinematic extract the rotation matrix that links **base_link** to **gripper_link** and then extract the only rotation component of the last three joints

In [39]:
# Orientation from the base_link to the gripper_link
# base_link to link_1
R0_1 = Matrix([[            cos(q1),              -sin(q1),                0],
               [sin(q1)*cos(alpha0),    cos(q1)*cos(alpha0),    -sin(alpha0)],
               [sin(q1)*sin(alpha0),    cos(q1)*sin(alpha0),     cos(alpha0)]])

# link_1 to link_2
R1_2 = Matrix([[            cos(q2),              -sin(q2),                0],
               [sin(q2)*cos(alpha1),    cos(q2)*cos(alpha1),    -sin(alpha1)],
               [sin(q2)*sin(alpha1),    cos(q2)*sin(alpha1),     cos(alpha1)]])

# link_2 to link_3
R2_3 = Matrix([[            cos(q3),              -sin(q3),                0],
               [sin(q3)*cos(alpha2),    cos(q3)*cos(alpha2),    -sin(alpha2)],
               [sin(q3)*sin(alpha2),    cos(q3)*sin(alpha2),     cos(alpha2)]])

# link_3 to link_4
R3_4 = Matrix([[            cos(q4),              -sin(q4),                0],
               [sin(q4)*cos(alpha3),    cos(q4)*cos(alpha3),    -sin(alpha3)],
               [sin(q4)*sin(alpha3),    cos(q4)*sin(alpha3),     cos(alpha3)]])

# link_4 to link_5
R4_5 = Matrix([[            cos(q5),              -sin(q5),                0],
               [sin(q5)*cos(alpha4),    cos(q5)*cos(alpha4),    -sin(alpha4)],
               [sin(q5)*sin(alpha4),    cos(q5)*sin(alpha4),     cos(alpha4)]])

# link_5 to link_6
R5_6 = Matrix([[            cos(q6),              -sin(q6),                0],
               [sin(q6)*cos(alpha5),    cos(q6)*cos(alpha5),    -sin(alpha5)],
               [sin(q6)*sin(alpha5),    cos(q6)*sin(alpha5),     cos(alpha5)]])


R0_3 = R0_1 * R1_2 * R2_3
R0_6 = R0_3 * R3_4 * R4_5 * R5_6

R3_6 = R0_3.inv(method="LU") * Rrpy

Extract the last three joint angles values

In [42]:
q4 = atan2(R3_6[1, 2], R3_6[0, 2])
q5 = atan2(sqrt(R3_6[0, 2]**2 + R3_6[1, 2]**2), R3_6[2,2])
q6 = atan2(-R3_6[2, 1], R3_6[2, 0])